In [ ]:
# Memory usage
#
# Get trace data using the provided launch file:
#    $ ros2 launch tracetools_analysis memory_usage.launch.py
#    (wait a few seconds, then kill with Ctrl+C)
# AND
# Convert trace data:
#    $ ros2 trace-analysis convert ~/.ros/tracing/memory-usage/ust

In [ ]:
converted_file_path = '~/.ros/tracing/memory-usage/ust/converted'

In [ ]:
import sys
# Assuming a workspace with:
#   src/tracetools_analysis/
#   src/micro-ROS/ros_tracing/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../micro-ROS/ros_tracing/ros2_tracing/tracetools_read/')
import datetime as dt

from bokeh.palettes import viridis
from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import NumeralTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor import Processor
from tracetools_analysis.processor.memory_usage import MemoryUsageHandler
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.memory_usage import MemoryUsageDataModelUtil
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [ ]:
# Process
events = load_file(converted_file_path)
memory_handler = MemoryUsageHandler()
ros2_handler = Ros2Handler()
Processor(memory_handler, ros2_handler).process(events)

In [ ]:
memory_data_util = MemoryUsageDataModelUtil(memory_handler.data)
ros2_data_util = Ros2DataModelUtil(ros2_handler.data)

output_notebook()
psize = 650

In [ ]:
# Plot memory usage
memory_usage_dfs = memory_data_util.get_absolute_memory_usage_by_tid()
tids = ros2_data_util.get_tids()

colours = viridis(len(tids) + 1)
first_tid = min(tids)
starttime = memory_usage_dfs[first_tid].loc[:, 'timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M')
memory = figure(
    title='Userspace memory usage per thread/node',
    x_axis_label=f'time ({starttime})',
    y_axis_label='memory usage',
    plot_width=psize, plot_height=psize,
)

i_colour = 0
for tid, memory_usage_df in memory_usage_dfs.items():
    if tid not in tids:
        continue
    memory.line(
        x='timestamp',
        y='memory_usage',
        legend=str(tid) + ' ' + str(ros2_data_util.get_node_names_from_tid(tid)),
        line_width=2,
        source=ColumnDataSource(memory_usage_df),
        line_color=colours[i_colour],
    )
    i_colour += 1

memory.title.align = 'center'
memory.legend.label_text_font_size = '11px'
memory.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])
memory.yaxis[0].formatter = NumeralTickFormatter(format='0.0b')

show(memory)